In [ ]:
# Lancer dans le Terminal
# !mlflow server --host 127.0.0.1 --port 8080

^C


In [ ]:
import mlflow

# Définir où MLflow stockera les logs et runs
mlflow.set_tracking_uri("file:///E:/Formation_Data_Analystic/projet_MLOps_GAI/Projet_MLOps/mlflow/mlruns")



# Liste des modèles pour lesquels on crée des expériences
model_names = ["LogisticRegression", "RandomForest", "DecisionTree"]

# Créer ou récupérer une expérience pour chaque modèle
for model in model_names:
    mlflow.set_experiment(model)
    print(f"Expérience '{model}' prête !")

2025/10/09 23:07:25 INFO mlflow.tracking.fluent: Experiment with name 'LogisticRegression' does not exist. Creating a new experiment.


2025/10/09 23:07:25 INFO mlflow.tracking.fluent: Experiment with name 'RandomForest' does not exist. Creating a new experiment.
2025/10/09 23:07:25 INFO mlflow.tracking.fluent: Experiment with name 'DecisionTree' does not exist. Creating a new experiment.


Expérience 'LogisticRegression' prête !
Expérience 'RandomForest' prête !
Expérience 'DecisionTree' prête !
